# Data Science Unit 1 Sprint Challenge 2

## Data Wrangling

In this Sprint Challenge you will use data from [Gapminder](https://www.gapminder.org/about-gapminder/), a Swedish non-profit co-founded by Hans Rosling. "Gapminder produces free teaching resources making the world understandable based on reliable statistics."
- [Cell phones (total), by country and year](https://raw.githubusercontent.com/open-numbers/ddf--gapminder--systema_globalis/master/ddf--datapoints--cell_phones_total--by--geo--time.csv)
- [Population (total), by country and year](https://raw.githubusercontent.com/open-numbers/ddf--gapminder--systema_globalis/master/ddf--datapoints--population_total--by--geo--time.csv)
- [Geo country codes](https://github.com/open-numbers/ddf--gapminder--systema_globalis/blob/master/ddf--entities--geo--country.csv)

These two links have everything you need to successfully complete the Sprint Challenge!
- [Pandas documentation: Working with Text Data](https://pandas.pydata.org/pandas-docs/stable/text.html]) (one question)
- [Pandas Cheat Sheet](https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf) (everything else)

In [38]:
import pandas as pd
import numpy as np

## Part 0. Load data

You don't need to add or change anything here. Just run this cell and it loads the data for you, into three dataframes.

In [2]:
cell_phones_url = 'https://raw.githubusercontent.com/open-numbers/ddf--gapminder--systema_globalis/master/ddf--datapoints--cell_phones_total--by--geo--time.csv'
population_url = 'https://raw.githubusercontent.com/open-numbers/ddf--gapminder--systema_globalis/master/ddf--datapoints--population_total--by--geo--time.csv'
country_codes_url = 'https://raw.githubusercontent.com/open-numbers/ddf--gapminder--systema_globalis/master/ddf--entities--geo--country.csv'

cell_phones = pd.read_csv(cell_phones_url)
population = pd.read_csv(population_url)
geo_country_codes = pd.read_csv(country_codes_url).rename(columns={'country': 'geo', 'name': 'country'})

## Part 1. Join data

First, join the `cell_phones` and `population` dataframes (with an inner join on `geo` and `time`).

The resulting dataframe's shape should be: (8590, 4)

In [7]:
cell_phones.head()

,geo,time,cell_phones_total
0,abw,1960,0.0
1,abw,1965,0.0
2,abw,1970,0.0
3,abw,1975,0.0
4,abw,1976,0.0


In [8]:
population.head()

,geo,time,population_total
0,afg,1800,3280000
1,afg,1801,3280000
2,afg,1802,3280000
3,afg,1803,3280000
4,afg,1804,3280000


In [11]:
cell_phones_and_pop = pd.merge(cell_phones, population, on=['geo', 'time'], how='inner')

In [12]:
cell_phones_and_pop.shape

(8590, 4)

Then, select the `geo` and `country` columns from the `geo_country_codes` dataframe, and join with your population and cell phone data.

The resulting dataframe's shape should be: (8590, 5)

In [16]:
phones_and_pop_by_country = pd.merge(geo_country_codes[['geo', 'country']], cell_phones_and_pop, on='geo')

In [17]:
phones_and_pop_by_country.shape

(8590, 5)

## Part 2. Make features

Calculate the number of cell phones per person, and add this column onto your dataframe.

(You've calculated correctly if you get 1.220 cell phones per person in the United States in 2017.)

In [18]:
phones_and_pop_by_country['phone_per_person'] = (
    phones_and_pop_by_country['cell_phones_total']/phones_and_pop_by_country['population_total']
)

In [32]:
is_2017 = phones_and_pop_by_country['time'] == 2017
is_united_states = phones_and_pop_by_country['country'] == 'United States'

phones_and_pop_by_country.loc[is_2017 & is_united_states, 'phone_per_person']

8134    1.220125
Name: phone_per_person, dtype: float64

Modify the `geo` column to make the geo codes uppercase instead of lowercase.

In [34]:
phones_and_pop_by_country['geo'] = phones_and_pop_by_country['geo'].str.upper()

In [35]:
phones_and_pop_by_country['geo'].sample(5)

7307    SWZ
3972    JPN
4596    LIE
2233    DZA
2923    GIN
Name: geo, dtype: object

## Part 3. Process data

Use the describe function, to describe your dataframe's numeric columns, and then its non-numeric columns.

(You'll see the time period ranges from 1960 to 2017, and there are 195 unique countries represented.)

In [40]:
phones_and_pop_by_country.describe(include=np.number)

,time,cell_phones_total,population_total,phone_per_person
count,8590.000000,8.590000e+03,8.590000e+03,8590.000000
mean,1994.193481,9.004950e+06,2.983823e+07,0.279639
std,14.257975,5.573408e+07,1.161284e+08,0.454247
min,1960.000000,0.000000e+00,4.433000e+03,0.000000
25%,1983.000000,0.000000e+00,1.456148e+06,0.000000
50%,1995.000000,6.200000e+03,5.725062e+06,0.001564
75%,2006.000000,1.697652e+06,1.810581e+07,0.461149
max,2017.000000,1.474097e+09,1.409517e+09,2.490243


In [41]:
phones_and_pop_by_country.describe(exclude=np.number)

,geo,country
count,8590,8590
unique,195,195
top,GTM,Switzerland
freq,46,46


In 2017, what were the top 5 countries with the most cell phones total?

Your list of countries should have these totals:

| country | cell phones total |
|:-------:|:-----------------:|
|    ?    |     1,474,097,000 |
|    ?    |     1,168,902,277 |
|    ?    |       458,923,202 |
|    ?    |       395,881,000 |
|    ?    |       236,488,548 |



In [42]:
# This optional code formats float numbers with comma separators
pd.options.display.float_format = '{:,}'.format

In [46]:
phones_and_pop_by_country[['country', 'time', 'cell_phones_total']][is_2017]\
    .sort_values(by='cell_phones_total', ascending=False)\
    .head(25)

,country,time,cell_phones_total
1496,China,2017,"1,474,097,000.0"
3595,India,2017,"1,168,902,277.0"
3549,Indonesia,2017,"458,923,202.0"
8134,United States,2017,"395,881,000.0"
1084,Brazil,2017,"236,488,548.0"
6636,Russia,2017,"227,341,873.0"
3990,Japan,2017,"170,128,499.0"
718,Bangladesh,2017,"145,113,669.0"
5748,Nigeria,2017,"144,920,170.0"
6104,Pakistan,2017,"144,525,637.0"


2017 was the first year that China had more cell phones than people.

What was the first year that the USA had more cell phones than people?

In [61]:
phones_greater_than_people = (
    phones_and_pop_by_country['population_total'] < phones_and_pop_by_country['cell_phones_total']
)

phones_and_pop_by_country.loc[is_united_states & phones_greater_than_people].sort_values(by='phone_per_person')

,geo,country,time,cell_phones_total,population_total,phone_per_person
8131,USA,United States,2014,"355,500,000.0",317718779,1.118914031833164
8132,USA,United States,2015,"382,307,000.0",319929162,1.1949739048796058
8134,USA,United States,2017,"395,881,000.0",324459463,1.2201246847283354
8133,USA,United States,2016,"395,881,000.0",322179605,1.228758722948959


In [52]:
# Or, The problem could've been solved like:
phone_per_person_saturated = 1 < phones_and_pop_by_country['phone_per_person']

phones_and_pop_by_country.loc[is_united_states & phone_per_person_saturated]

,geo,country,time,cell_phones_total,population_total,phone_per_person
8131,USA,United States,2014,"355,500,000.0",317718779,1.118914031833164
8132,USA,United States,2015,"382,307,000.0",319929162,1.1949739048796058
8133,USA,United States,2016,"395,881,000.0",322179605,1.228758722948959
8134,USA,United States,2017,"395,881,000.0",324459463,1.2201246847283354


In [62]:
df = phones_and_pop_by_country.loc[is_united_states & phone_per_person_saturated].sort_values(by='phone_per_person')
df['time'].head(1)

8131    2014
Name: time, dtype: int64

## Part 4. Reshape data

Create a pivot table:
- Columns: Years 2007—2017
- Rows: China, India, United States, Indonesia, Brazil (order doesn't matter)
- Values: Cell Phones Total

The table's shape should be: (5, 11)

In [90]:
is_or_after_2007 = 2007 <= phones_and_pop_by_country['time']
is_or_before_2017 = phones_and_pop_by_country['time'] <= 2017
between_2007_and_2017 = is_or_after_2007 & is_or_before_2017

countries = ['China', 'India', 'United States', 'Indonesia', 'Brazil']
one_of_us_cn_in_id_br = phones_and_pop_by_country['country'].isin(countries)

df = phones_and_pop_by_country[between_2007_and_2017 & one_of_us_cn_in_id_br]
phones_by_country_and_year = df.pivot(index='country', columns='time', values='cell_phones_total')

phones_by_country_and_year

time,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
country,,,,,,,,,,,
Brazil,"120,980,103.0","150,641,403.0","169,385,584.0","196,929,978.0","234,357,507.0","248,323,703.0","271,099,799.0","280,728,796.0","257,814,274.0","244,067,356.0","236,488,548.0"
China,"547,306,000.0","641,245,000.0","747,214,000.0","859,003,000.0","986,253,000.0","1,112,155,000.0","1,229,113,000.0","1,286,093,000.0","1,291,984,200.0","1,364,934,000.0","1,474,097,000.0"
India,"233,620,000.0","346,890,000.0","525,090,000.0","752,190,000.0","893,862,478.0","864,720,917.0","886,304,245.0","944,008,677.0","1,001,056,000.0","1,127,809,000.0","1,168,902,277.0"
Indonesia,"93,386,881.0","140,578,243.0","163,676,961.0","211,290,235.0","249,805,619.0","281,963,665.0","313,226,914.0","325,582,819.0","338,948,340.0","385,573,398.0","458,923,202.0"
United States,"249,300,000.0","261,300,000.0","274,283,000.0","285,118,000.0","297,404,000.0","304,838,000.0","310,698,000.0","355,500,000.0","382,307,000.0","395,881,000.0","395,881,000.0"


In [91]:
phones_by_country_and_year.shape

(5, 11)

#### OPTIONAL BONUS QUESTION!

Sort these 5 countries, by biggest increase in cell phones from 2007 to 2017.

Which country had 935,282,277 more cell phones in 2017 versus 2007?

In [97]:
phones_by_country_and_year['2007-2017 Phone Increase'] = (
    phones_by_country_and_year[2017]/phones_by_country_and_year[2007]
)

sorted_phone_change = phones_by_country_and_year.sort_values(by='2007-2017 Phone Increase', ascending=False)
sorted_phone_change.head(1)[[2007, 2017, '2007-2017 Phone Increase']]

time,2007,2017,2007-2017 Phone Increase
country,,,
India,"233,620,000.0","1,168,902,277.0",5.003434110949405


If you have the time and curiosity, what other questions can you ask and answer with this data?